In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd

In [ ]:
import typing
from typing import List, Tuple, Dict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Colab-Specific Settings

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# Config

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
colab_prefix = "drive/MyDrive/CMPUT651_DL4NLP/"

source_embedding_dim = 300
model_type = "supervised_distilled"
version = 1
embedding_dim = 300
nn_hidden_size = 50

experiment_name = f"{model_type}_{source_embedding_dim}to{embedding_dim}_v{version}"
embedding_path = colab_prefix + f"data/embeddings/trained/{experiment_name}.glove.6B.300d.txt"
model_output_path = colab_prefix + f"models/ag_news_classifier_glove_{experiment_name}.pt"
# When using standard embeddings
# embedding_path = colab_prefix + f"data/embeddings/base/clipped.glove.6B.{embedding_dim}d.txt"
# model_output_path = colab_prefix + f"models/ag_news_classifier_glove_clipped_{embedding_dim}d.pt"

freeze_embeddings = True
epochs = 5
batch_size = 32
learning_rate = 1e-3

train_datapath = colab_prefix + "data/datasets/ag_news/train.csv"
test_datapath = colab_prefix + "data/datasets/ag_news/test.csv"

pad_tag = "<PAD>"
unk_tag = "<UNK>"

In [ ]:
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")

In [ ]:
# print(experiment_name)
print(gpu, device)

True cuda


# Load Embeddings

In [ ]:
words = []
vectors = []
with open(embedding_path, "r", encoding="utf-8") as fp:
    for line in fp:
        line = line.split()
        word = line[0]
        vector = np.asarray(line[1:], dtype='float32')
        words.append(word)
        vectors.append(vector)
vectors = np.asarray(vectors)

Create an embedding for both \<PAD> (all 0s) and \<UNK> (average of all embeddings) tags.

In [ ]:
unk_embedding = np.mean(vectors, axis=0).reshape(1, -1)
pad_embedding = np.zeros((1, vectors.shape[1]))

In [ ]:
vectors = torch.as_tensor(np.concatenate((vectors, pad_embedding, unk_embedding)))

Set up dictionaries for converting tags to indices, tokens to indices and vice-versa.

In [ ]:
token2index = {word: i for i, word in enumerate(words)}
pad_token_index = len(token2index)
unk_token_index = len(token2index) + 1
token2index[pad_tag] = pad_token_index
token2index[unk_tag] = unk_token_index

index2token = {i: word for word, i in token2index.items()}

In [ ]:
index2tag ={
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

# Load Data & Preprocess

In [ ]:
def sentence_to_indices(sentence, token2index, unk_token_index):
    return [token2index.get(word, unk_token_index) for word in sentence]

In [ ]:
def indices_to_sentence(sentence, index2token):
    return [index2token[int(index)] for index in sentence]

In [ ]:
def pad_to_max_length(sentence, pad_token_index, max_length):
    padding = [pad_token_index] * (max_length - len(sentence))
    return sentence + padding

In [ ]:
def preprocess_data(df, token2index, pad_token_index, unk_token_index, title_max_length=0, desc_max_length=0):
    print("Splitting & Lowercasing.")
    df['Title'] = df['Title'].str.lower().str.split()
    df['Description'] = df['Description'].str.lower().str.split()
    
    # If no max length is specified, compute from data
    if (title_max_length == desc_max_length) and (title_max_length == 0):
        print("Computing Max Lengths.")
        title_max_length = df['Title'].apply(len).max()
        desc_max_length = df['Description'].apply(len).max()
    
    # Convert tokens to indices
    print("Transforming tokens into indices.")
    df['Title'] = df['Title'].apply(sentence_to_indices, args=(token2index, unk_token_index))
    df['Description'] = df['Description'].apply(sentence_to_indices, args=(token2index, unk_token_index))
    
    # Pad data
    print("Padding data.")
    df['Title'] = df['Title'].apply(pad_to_max_length, args=(pad_token_index, title_max_length))
    df['Description'] = df['Description'].apply(pad_to_max_length, args=(pad_token_index, desc_max_length))
    
    # Convert to tensor
    print("Splitting features & labels and converting to tensors.")
    data = df.to_dict(orient="records")
    titles = [x['Title'] for x in data]
    descriptions = [x['Description'] for x in data]
    features = [torch.as_tensor(titles), torch.as_tensor(descriptions)]
    labels = torch.as_tensor([x['Class Index'] for x in data]) - 1 # We want 0-3 not 1-4
    return features, labels, title_max_length, desc_max_length

In [ ]:
train = pd.read_csv(train_datapath)
test = pd.read_csv(test_datapath)

In [ ]:
train, val = train_test_split(
    train, test_size=4000, stratify=train['Class Index'], random_state=seed
)

In [ ]:
len(train), len(val), len(test)

(116000, 4000, 7600)

In [ ]:
train_features, train_labels, title_max_length, desc_max_length = preprocess_data(
    train, token2index, pad_token_index, unk_token_index
)

Splitting & Lowercasing.
Computing Max Lengths.
Transforming tokens into indices.
Padding data.
Splitting features & labels and converting to tensors.


In [ ]:
val_features, val_labels, title_max_length, desc_max_lenght = preprocess_data(
    val, token2index, pad_token_index, unk_token_index, title_max_length, desc_max_length
)

Splitting & Lowercasing.
Transforming tokens into indices.
Padding data.
Splitting features & labels and converting to tensors.


In [ ]:
test_features, test_labels, title_max_length, desc_max_lenght = preprocess_data(
    test, token2index, pad_token_index, unk_token_index, title_max_length, desc_max_length
)

Splitting & Lowercasing.
Transforming tokens into indices.
Padding data.
Splitting features & labels and converting to tensors.


In [ ]:
title_max_length, desc_max_length

(19, 173)

In [ ]:
num_classes = len(train_labels.unique())

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_features[0], train_features[1], train_labels)
val_dataset = torch.utils.data.TensorDataset(val_features[0], val_features[1], val_labels)
test_dataset = torch.utils.data.TensorDataset(test_features[0], test_features[1], test_labels)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=gpu, 
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=gpu, 
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=gpu, 
)

# Model Time

In [ ]:
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
class BiLSTM(nn.Module):
    def __init__(self, embeddings, embedding_dim, hidden_dim, num_classes, freeze_embeddings):
        super(BiLSTM, self).__init__()
        self.word_embeddings = nn.Embedding.from_pretrained(embeddings, freeze=freeze_embeddings)
        self.title_lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.desc_lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(hidden_dim * 4, num_classes) # Since BiLSTM + 2 inputs

    def forward(self, title, description):
        # (batch_size, seq_len) -> (batch_size, seq_len, embedding_dim)
        title_embedding = self.word_embeddings(title)
        desc_embedding = self.word_embeddings(description)
        # (batch_size, seq_len, embedding_dim) -> (batch_size, directions, hidden_dim) 
        _, (title_hidden, _) = self.title_lstm(title_embedding) 
        _, (desc_hidden, _) = self.desc_lstm(desc_embedding)
        # (directions, batch_size, hidden_dim), (directions, batch_size, hidden_dim)
        # -> (directions, batch_size, 2*hidden_dim)
        out = torch.cat((title_hidden, desc_hidden), dim=2)
        # (batch_size, directions, 2*hidden_dim) -> (batch_size, 2*hidden_dim*directions)
        out = out.permute(1, 0, 2).flatten(start_dim=1)
        # (batch_size, 2*hidden_dim*directions) -> (batch_size, num_classes)|
        out = self.classifier(out)
        # We use the CrossEntropyLoss so we aren't adding a softmax layer here
        # Because in PyTorch CrossEntropyLoss combines a LogSoftmax with NLLLoss
        # So we output raw logits
        # Since we don't care about the confidences, we don't need a softmax during inference
        # Since the highest value in a softmax will always be the highest value pre-softmax
        return out

In [ ]:
model = BiLSTM(vectors, embedding_dim, nn_hidden_size, num_classes, freeze_embeddings)
model.double() # Since our embeddings are 32-dimensional
model.to(device)
loss_function = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model

BiLSTM(
  (word_embeddings): Embedding(400002, 300)
  (title_lstm): LSTM(300, 50, batch_first=True, bidirectional=True)
  (desc_lstm): LSTM(300, 50, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=200, out_features=4, bias=True)
)

# Training

In [ ]:
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    
    # Training Loop
    for iteration, batch in enumerate(tqdm(train_dataloader)):
        # Move data to device
        titles, descriptions, labels = batch
        titles = titles.to(device)
        descriptions = descriptions.to(device)
        labels = labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(titles, descriptions)
        
        # Calculate loss
        batch_loss = loss_function(predictions, labels)
        
        # Backward pass
        batch_loss.backward()
        optimizer.step()
        
        # Update train loss
        train_loss += batch_loss.item()
    
    # Validation Loop
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(val_dataloader)):
            # Move data to device
            titles, descriptions, labels = batch
            titles = titles.to(device)
            descriptions = descriptions.to(device)
            labels = labels.to(device)

            # Forward pass
            predictions = model(titles, descriptions)

            # Calculate loss
            batch_loss = loss_function(predictions, labels)

            # Update train loss
            val_loss += batch_loss.item()
    
    # Compute the average losses for this epoch
    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    
    
    # Print Metrics
    print(
        f"Epoch: {epoch+1}/{epochs}, Train Loss = {train_loss}, \
        Validation Loss = {val_loss}"
    )

  0%|          | 0/3625 [00:00<?, ?it/s]

Epoch: 1/5, Train Loss = 0.30527777847563003,         Validation Loss = 0.253713835842478


  0%|          | 0/3625 [00:00<?, ?it/s]

Epoch: 2/5, Train Loss = 0.2243793340084654,         Validation Loss = 0.23641120083268766


  0%|          | 0/3625 [00:00<?, ?it/s]

Epoch: 3/5, Train Loss = 0.18909123871016834,         Validation Loss = 0.23164164714414195


  0%|          | 0/3625 [00:00<?, ?it/s]

Epoch: 4/5, Train Loss = 0.15695351538103852,         Validation Loss = 0.23597222452108257


100%|██████████| 125/125 [00:01<00:00, 71.29it/s]

Epoch: 5/5, Train Loss = 0.12826177359975216,         Validation Loss = 0.2446062633396359


# Evaluation

In [ ]:
model.eval()

BiLSTM(
  (word_embeddings): Embedding(400002, 300)
  (title_lstm): LSTM(300, 50, batch_first=True, bidirectional=True)
  (desc_lstm): LSTM(300, 50, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=200, out_features=4, bias=True)
)

In [ ]:
# Test Loop
with torch.no_grad():
    y_true = []
    y_pred = []
    for iteration, batch in enumerate(tqdm(test_dataloader)):
        # Move data to device
        titles, descriptions, labels = batch
        titles = titles.to(device)
        descriptions = descriptions.to(device)
        labels = labels.to(device)

        # Forward pass
        predictions = model(titles, descriptions).detach().cpu().numpy().argmax(axis=1)
        
        y_true.extend(labels.detach().cpu().numpy())
        y_pred.extend(predictions)
y_true = indices_to_sentence(y_true, index2tag)
y_pred = indices_to_sentence(y_pred, index2tag)

100%|██████████| 238/238 [00:03<00:00, 76.47it/s]


In [ ]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

    Business     0.8898    0.8800    0.8849      1900
    Sci/Tech     0.8838    0.9047    0.8941      1900
      Sports     0.9690    0.9705    0.9698      1900
       World     0.9306    0.9174    0.9239      1900

    accuracy                         0.9182      7600
   macro avg     0.9183    0.9182    0.9182      7600
weighted avg     0.9183    0.9182    0.9182      7600



# Save

In [ ]:
torch.save(model.state_dict(), model_output_path)